In [1]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from covid19model.data import sciensano
from covid19model.models.utils import initialize_COVID19_SEIQRD_hybrid_vacc

# OPTIONAL: Load the "autoreload" extension so that package code can change
%load_ext autoreload
# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
%autoreload 2

In [2]:
# Number of age groups used in the model
age_stratification_size=10
# Startdate of model
start_date = '2020-03-15'

In [3]:
#model, BASE_samples_dict, initN = initialize_COVID19_SEIQRD_hybrid_vacc(age_stratification_size=age_stratification_size, start_date=start_date, update_data=False)

In [70]:
agg=None
df = sciensano.get_public_spatial_vaccination_data(agg=agg, update=False)
df = df['INCIDENCE']

In [122]:
from functools import lru_cache

class make_N_vacc_function():
    """A time-dependent parameter function to return the vaccine incidence at each timestep of the simulation
       TODO: hypothetical vaccination schemes
    """
    
    def __init__(self, agg=None, df_incidences=None, age_classes=pd.IntervalIndex.from_tuples([(0,12),(12,18),(18,25),(25,35),(35,45),(45,55),(55,65),(65,75),(75,85),(85,120)], closed='left')):
        
        #####################
        ## Input variables ##
        #####################
        
        # Window length of exponential moving average to smooth vaccine incidence
        filter_length = 14
        
        #################
        ## Input check ##
        #################
        
        df = df_incidences
        if not (df.index.get_level_values('dose').unique() == ['A', 'B', 'C', 'E']).all():
            # This code was built at a point in time when the following doses were in the raw dataset provided by Sciensano:
            # A: 0 --> P, B: P --> F, C: 0 --> F, E: F --> B
            # If you get this error it means Sciensano has changed the definition of these doses
            raise ValueError("The names of the 'doses' in the vaccination dataset have been changed from those the code was designed for ({0})".format(df.index.get_level_values('dose').unique().values))
        
        ####################################
        ## Step 1: Perform age conversion ##
        ####################################
        
        # Only perform age conversion if necessary
        if len(age_classes) != len(df.index.get_level_values('age').unique()):
            df = self.age_conversion(df, age_classes, agg)
        elif (age_classes != df.index.get_level_values('age').unique()).any():
            df = self.age_conversion(df, age_classes, agg)
        
        #########################################################
        ## Step 2: Hypothetical vaccination campaigns go here? ##
        #########################################################
        
        weekly_doses = 7*60000
        vacc_order = range(len(age_classes))[::-1]
        stop_idx = -1
        refusal = 0.1*np.ones(len(age_classes))
        extend_weeks = 12
        out = self.append_booster_campaign(df, extend_weeks, weekly_doses, vacc_order, stop_idx, refusal)
        
        #####################################################################
        ## Step 3: Convert weekly to daily incidences and smooth using EMA ##
        #####################################################################

        df = self.smooth_incidences(df, filter_length, agg)
        
        ##################################################
        ## Step 4: Assign relevant (meta)data to object ##
        ##################################################
        
        # Dataframe 
        self.df = df
        # Number of spatial patches
        try:
            self.G = len(self.df.index.get_level_values('NIS').unique().values)
        except:
            pass
        # Aggregation    
        self.agg = agg
        # Number of age groups
        self.N = len(age_classes)
        # Number of vaccine doses
        self.D = len(self.df.index.get_level_values('dose').unique().values)
        
        return None
        
    @lru_cache()
    def get_data(self,t):
        if self.agg:
            try:
                return np.array(self.df.loc[t,:,:,:].values).reshape( (self.G, self.N, self.D) )
            except:
                return np.zeros([self.G, self.N, self.D])
        else:
            try:
                return np.array(self.df.loc[t,:,:].values).reshape( (self.N, self.D) )
            except:
                return np.zeros([self.N, self.D])
    
    def __call__(self, t, states, param):
        """
        Time-dependent parameter function compatible wrapper for cached function `get_data`
        Returns the vaccination incidence at time "t"
        
        Input
        -----
        
        t : pd.Timestamp
            Current date in simulation
        
        Returns
        -------
        
        N_vacc : np.array
            Number of individuals to be vaccinated at simulation time "t" per [age, (space), dose]
        """

        return self.get_data(t)
    
    ######################
    ## Helper functions ##
    ######################
    
    @staticmethod
    def smooth_incidences(df, filter_length, agg):
        
        # Start- and enddate
        df_start = pd.Timestamp(df.index.get_level_values('date').min())
        df_end = pd.Timestamp(df.index.get_level_values('date').max())
    
        # Make a dataframe with the desired format
        iterables=[]
        for index_name in df.index.names:
            if index_name != 'date':
                iterables += [df.index.get_level_values(index_name).unique()]
            else:
                iterables += [pd.date_range(start=df_start, end=df_end, freq='D'),]
        index = pd.MultiIndex.from_product(iterables, names=df.index.names)
        df_new = pd.Series(index=index, name = df.name, dtype=float)

        # Loop over (NIS),age,dose
        for age in df.index.get_level_values('age').unique():
            for dose in df.index.get_level_values('dose').unique():
                if agg:
                    for NIS in df.index.get_level_values('NIS').unique():
                         # Convert weekly to daily data
                        daily_data = df.loc[slice(None), NIS, age, dose].resample('D').bfill().apply(lambda x : x/7)
                        # Apply an exponential moving average
                        daily_data_EMA = daily_data.ewm(span=filter_length, adjust=False).mean()
                        # Assign to new dataframe
                        df_new.loc[slice(None), NIS, age, dose] = daily_data_EMA.values
                else:
                    # Convert weekly to daily data
                    daily_data = df.loc[slice(None), age, dose].resample('D').bfill().apply(lambda x : x/7)
                    # Apply an exponential moving average
                    daily_data_EMA = daily_data.ewm(span=filter_length, adjust=False).mean()
                    # Assign to new dataframe
                    df_new.loc[slice(None), age, dose] = daily_data_EMA.values
                    
        return df_new
        
    @staticmethod
    def age_conversion(df, desired_age_classes, agg):
        
        from covid19model.data.utils import convert_age_stratified_quantity
        
        # Define a new dataframe with the desired age groups
        iterables=[]
        for index_name in df.index.names:
            if index_name != 'age':
                iterables += [df.index.get_level_values(index_name).unique()]
            else:
                iterables += [desired_age_classes]
        index = pd.MultiIndex.from_product(iterables, names=df.index.names)
        df_new = pd.Series(index=index, dtype=float)
        
        # Loop to the dataseries level and perform demographic aggregation
        if agg:
            for date in df.index.get_level_values('date').unique():
                for NIS in df.index.get_level_values('NIS').unique():
                    for dose in df.index.get_level_values('dose').unique():
                        data = df.loc[date, NIS, slice(None), dose].reset_index().set_index('age')
                        df_new.loc[date, NIS, slice(None), dose] = convert_age_stratified_quantity(data, desired_age_classes, agg=agg, NIS=NIS).values
        else:
            for date in df.index.get_level_values('date').unique():
                for dose in df.index.get_level_values('dose').unique():
                    df_new.loc[date, slice(None), dose] = convert_age_stratified_quantity(data, desired_age_classes).values

        return df_new
    
    @staticmethod
    def append_booster_campaign(df, extend_weeks, daily_doses, vacc_order, stop_idx, refusal):

        # Specify dose
        desired_dose = 'E'
        
        # Compute the cumulative number of fully vaccinated at last timestep
        cumulative_fully_vaccinated=[]
        cumulative_boosted=[]
        for age_class in df.index.get_level_values('age').unique():
            cumulative_fully_vaccinated.append((df.loc[slice(None), age_class, 'B'] + df.loc[slice(None), age_class, 'C']).cumsum().iloc[-1])

        print(cumulative_fully_vaccinated)
        
        # Make a dataframe with the desired format
        df_end = pd.Timestamp(df.index.get_level_values('date').max())
        iterables=[]
        for index_name in df.index.names:
            if index_name != 'date':
                iterables += [df.index.get_level_values(index_name).unique()]
            else:
                iterables += [pd.date_range(start=df_end, end=df_end+pd.Timedelta(days=extend_weeks*7), freq='W-MON'),]
        index = pd.MultiIndex.from_product(iterables, names=df.index.names)
        df_new = pd.Series(index=index, name = df.name, dtype=float)#.fillna(0)
        
        # Loop over the dates
        for date in df_new.index.get_level_values('date').unique():
            # Initialize N_vacc
            N_vacc = np.zeros([self.N,self.D])
            # 
            
        print(df_new)
        return df_new

In [123]:
output = make_N_vacc_function(agg=agg, df_incidences=df)

[474097.6693652556, 457578.3306347444, 767935.0, 1219810.0, 1276177.0, 1378747.0, 1416384.0, 1116259.0, 658568.0, 299370.0]


NameError: name 'self' is not defined

In [81]:
output(t, {}, {})

array([[1.33442746e+01, 1.88105610e+01, 2.86236695e-07, 3.14708386e+01],
       [1.28035555e+01, 1.78645770e+01, 2.65963538e-07, 3.02905713e+01],
       [4.53210727e+00, 3.87674087e+00, 3.06644006e-02, 2.82264878e+01],
       [7.22879261e+00, 6.33955054e+00, 1.41252198e-01, 4.96726831e+01],
       [6.60440651e+00, 5.27343258e+00, 1.64619438e-01, 3.58241863e+01],
       [3.41466226e+00, 3.11453466e+00, 3.18696903e-02, 2.69380588e+01],
       [2.16723863e+00, 2.61454220e+00, 6.68258961e-02, 2.14989564e+01],
       [1.40264881e+00, 1.40487135e+00, 2.27786855e-02, 1.13872113e+01],
       [9.20857575e-01, 6.17278618e-01, 1.86075711e-06, 5.49786360e+00],
       [1.21791854e-01, 1.59068120e-01, 6.47842248e-08, 2.05105313e+00]])